In [2]:
pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 3.1 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [34]:
import pandas as pd

# Replace 'your_file.xlsx' with the path to your Excel file
file_path = 'dataframe_input.xlsx'

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Now 'df' contains your Excel data in a DataFrame format
df.head(10)

,dish_id,path_filename,ingr_1_name,ingr_2_name,ingr_3_name,ingr_4_name,ingr_5_name
0,dish_1556572657,\dish_1556572657\rgb.png,olives,NaN,NaN,NaN,NaN
1,dish_1556573514,\dish_1556573514\rgb.png,mixed greens,NaN,NaN,NaN,NaN
2,dish_1556575014,\dish_1556575014\rgb.png,olives,NaN,NaN,NaN,NaN
3,dish_1556575083,\dish_1556575083\rgb.png,brussels sprouts,NaN,NaN,NaN,NaN
4,dish_1556575124,\dish_1556575124\rgb.png,celery,NaN,NaN,NaN,NaN
5,dish_1561737271,\dish_1561737271\rgb.png,sausage,hash browns,pineapple,broccoli,egg whites
6,dish_1561737293,\dish_1561737293\rgb.png,broccoli,cantaloupe,sausage,pineapple,egg whites
7,dish_1561737776,\dish_1561737776\rgb.png,bagels,cream cheese,NaN,NaN,NaN
8,dish_1561737839,\dish_1561737839\rgb.png,berries,NaN,NaN,NaN,NaN
9,dish_1561739160,\dish_1561739160\rgb.png,berries,NaN,NaN,NaN,NaN


In [35]:
df.columns

Index(['dish_id', 'path_filename', 'ingr_1_name', 'ingr_2_name', 'ingr_3_name',
       'ingr_4_name', 'ingr_5_name'],
      dtype='object')

In [36]:
from sklearn.preprocessing import MultiLabelBinarizer

# Handle Missing Values
df.fillna('01_erase_me', inplace=True)

# Tokenize Ingredients
ingredients = df[['ingr_1_name', 'ingr_2_name', 'ingr_3_name', 'ingr_4_name', 'ingr_5_name']].values.tolist()

# Flatten list of ingredients
all_ingredients = [ingredient for sublist in ingredients for ingredient in sublist]

# Prepare Labels
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(ingredients)

# Add encoded labels to DataFrame
for i, label in enumerate(mlb.classes_):
    df[label] = labels[:, i]

# Drop original ingredient columns
df.drop(columns=['ingr_1_name', 'ingr_2_name', 'ingr_3_name', 'ingr_4_name', 'ingr_5_name'], inplace=True)

# You can now use df for your deep learning model
df.head(10)

,dish_id,path_filename,01_erase_me,bagels,berries,broccoli,brussels sprouts,cantaloupe,celery,cream cheese,egg whites,hash browns,mixed greens,olives,pineapple,sausage
0,dish_1556572657,\dish_1556572657\rgb.png,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,dish_1556573514,\dish_1556573514\rgb.png,1,0,0,0,0,0,0,0,0,0,1,0,0,0
2,dish_1556575014,\dish_1556575014\rgb.png,1,0,0,0,0,0,0,0,0,0,0,1,0,0
3,dish_1556575083,\dish_1556575083\rgb.png,1,0,0,0,1,0,0,0,0,0,0,0,0,0
4,dish_1556575124,\dish_1556575124\rgb.png,1,0,0,0,0,0,1,0,0,0,0,0,0,0
5,dish_1561737271,\dish_1561737271\rgb.png,0,0,0,1,0,0,0,0,1,1,0,0,1,1
6,dish_1561737293,\dish_1561737293\rgb.png,0,0,0,1,0,1,0,0,1,0,0,0,1,1
7,dish_1561737776,\dish_1561737776\rgb.png,1,1,0,0,0,0,0,1,0,0,0,0,0,0
8,dish_1561737839,\dish_1561737839\rgb.png,1,0,1,0,0,0,0,0,0,0,0,0,0,0
9,dish_1561739160,\dish_1561739160\rgb.png,1,0,1,0,0,0,0,0,0,0,0,0,0,0


In [38]:
df.drop(columns=['01_erase_me'], inplace = True)
df.head()

,dish_id,path_filename,bagels,berries,broccoli,brussels sprouts,cantaloupe,celery,cream cheese,egg whites,hash browns,mixed greens,olives,pineapple,sausage
0,dish_1556572657,\dish_1556572657\rgb.png,0,0,0,0,0,0,0,0,0,0,1,0,0
1,dish_1556573514,\dish_1556573514\rgb.png,0,0,0,0,0,0,0,0,0,1,0,0,0
2,dish_1556575014,\dish_1556575014\rgb.png,0,0,0,0,0,0,0,0,0,0,1,0,0
3,dish_1556575083,\dish_1556575083\rgb.png,0,0,0,1,0,0,0,0,0,0,0,0,0
4,dish_1556575124,\dish_1556575124\rgb.png,0,0,0,0,0,1,0,0,0,0,0,0,0


In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models


#Preprocessing:
'''Next, image augmentation techniques such as rotation, translation, and flipping can be applied to increase the diversity of the training data.
This helps the model to generalize better and perform well on unseen images.'''

# Assumptions:
# - `df_images` is a DataFrame with image paths and IDs
# - `df_ingredients` is a DataFrame with ingredients information and IDs
# - `your_label_columns` are the columns representing the ingredients information in the DataFrame

# Merge both DataFrames based on the image IDs
#merged_df = pd.merge(df_images, df_ingredients, on='image_id')

# Split the data into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Create data generators for images
'''train_datagen is used to generate data for training the model by reading images from a directory, performing transformations, and normalization.

ImageDataGenerator is a class in TensorFlow/Keras that generates batches of augmented/normalized data from image data.
It provides a flexible way to preprocess and augment images on-the-fly during training, without needing to pre-process and store all the images in memory.'''
train_datagen = ImageDataGenerator(rescale=1./255) #used below in train_generator
test_datagen = ImageDataGenerator(rescale=1./255)  #used below in test_generator


'''The train_generator generates batches of augmented/normalized data from image data and labels in the form of a DataFrame.
In the provided code, the train_generator is specifically responsible for generating training data for the model.
It generates batches of data from the DataFrame train_df, which contains information about image paths and corresponding labels (ingredients in this case).'''

#VARIABLES in train_generator:
'''The "image_path" parameter in the flow_from_dataframe() method indicates which column of the DataFrame contains the paths to the images.
When you provide this parameter, the ImageDataGenerator will use the paths specified in that column to load the images for processing.'''

'''your_label_columns represents the column(s) in your DataFrame that contain the labels or target variables for your classification task.
In the context of the provided code, it indicates the column(s) that contain information about the ingredients or classes associated with each image.'''

'''batch_size is a hyperparameter that determines the number of samples processed by the model in each training iteration or batch during the training process.'''

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col=train_df['path_filename'],
    y_col=train_df.drop(columns=['dish_id', 'path_filename']),
    target_size=(640, 480),
    batch_size=10,
    class_mode='raw'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col=test_df['path_filename'],
    y_col=test_df.drop(columns=['dish_id', 'path_filename']),
    target_size=(640, 480),
    batch_size=10,
    class_mode='raw'
)



'''num_channels typically refers to the number of channels or color channels in an image.
In the context of the provided code, num_channels specifies the number of color channels for the input images. For example:
For grayscale images, num_channels would be 1.
For RGB (Red, Green, Blue) color images, num_channels would be 3, as there are three color channels representing the intensity of each color.'''

#Modified for multi-label classification:
'''Each node in the output layer must use the SIGMOID activation.
This will predict a probability of class membership for the label, a value between 0 and 1.
Finally, the model must be fit with the BINARY CROSS-ENTROPY loss function.
https://machinelearningmastery.com/multi-label-classification-with-deep-learning/'''

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(640, 480, 3), padding='same'), #32 kernels with size 3,3
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),

    layers.Dense(64, activation='relu'),
    layers.Dense(5, activation='sigmoid')           #num_classes is = number of ingredients
])

'''This layer performs max pooling, which reduces the spatial dimensions of the feature maps while retaining the most important information.
(2, 2) specifies the size of the pooling window, which determines how much the image is downsampled.'''
'''These layers further extract features from the input image.
Each subsequent convolutional layer learns more complex and abstract features from the previous layer's output.'''
'''his layer flattens the 2D feature maps into a 1D vector.
It prepares the data for input into the fully connected (dense) layers.'''

'''Dense: These layers are fully connected layers that perform classification based on the features extracted by the convolutional layers.
The first Dense layer has 64 units, meaning it has 64 neurons. It applies the ReLU activation function to introduce non-linearity.
The second Dense layer has num_classes units, which corresponds to the number of output classes (ingredients).
It uses the sigmoid activation function to output probabilities for each class, as specified.'''

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=test_generator)

KeyError: "None of [Index(['ingr_1_name', 'ingr_2_name', 'ingr_3_name', 'ingr_4_name',\n       'ingr_5_name'],\n      dtype='object')] are in the [columns]"